# Api Get Items Data

In [ ]:
# Imports

import pandas as pd
import csv
import requests
import re
import json

In [ ]:
# Read credentials

with open('../api_credentials.txt', 'r') as file:
   text = file.read()
    
credentials = re.findall('=(.*)', text)

In [ ]:
# Assign credentials

client_ID = credentials[0]
client_secret = credentials[1]

In [ ]:
# Read unique item ids

with open('../data/item_ids.csv', mode = 'r') as file:

   reader = csv.reader(file)
   item_ids = [int(row[0]) for row in reader]

In [ ]:
# Check amount ot items

len(item_ids)

In [ ]:
# Get items data from Api

def get_item(ids):
   items = []
   failed_requests = []
   count = 0

   # Recreate access token due to Api restrictions
   if count % 1000 == 0:
      def create_access_token(client_ID, client_secret, region = 'eu'):
         data = {'grant_type': 'client_credentials'}
         response = requests.post('https://%s.battle.net/oauth/token' % region, data = data, auth = (client_ID, client_secret))
         return response.json()

      response = create_access_token(client_ID, client_secret)
      token = response['access_token']
   
   # Loop Api calls through item_id list
   for item_id in ids:
      try:
         # Api call
         search = f'https://eu.api.blizzard.com/data/wow/item/{item_id}?namespace=static-eu&access_token={token}'
         response = requests.get(search)
         item_body = response.json()
         item_object = {}
         # Handle 404 items
         if 'code' in item_body.keys():
            if item_body['code'] == 404:
               items.append({'ID': item_id, 'NAME': 'UNKNOWN', 'QUALITY': 'UNKNOWN', 'LEVEL': 0, 'ITEM CLASS': 'UNKNOWN', 'ITEM SUBCLASS': 'UNKNOWN', 'DESCRIPTION': 'UNKNOWN'})
         else:
            # Transform item objects
            item_object['item_id'] = item_id
            item_object['name'] = item_body['name']['en_GB']
            item_object['quality'] = item_body['quality']['name']['en_GB']
            item_object['level'] = item_body['level']
            item_object['item_class'] = item_body['item_class']['name']['en_GB']
            item_object['item_subclass'] = item_body['item_subclass']['name']['en_GB']
            if 'description' in item_body:
               item_object['description'] = re.sub(r'\s+', ' ', item_body['description']['en_GB'])
            elif 'spells' in item_body['preview_item']:
               item_object['description'] = re.sub(r'\s+', ' ', item_body['preview_item']['spells'][0]['description']['en_GB'])
            elif len(re.findall('description', json.dumps(item_body))) > 0:
               item_object['description'] = 'CHECK description'
            else:
               item_object['description'] = 'no description'
            # Build list of items
            items.append(item_object)

         # Check progress
         if count % 1000 == 0:   
            print('Line:', count, 'Item ID:', item_id)
         count += 1

      # Handle errors
      except json.decoder.JSONDecodeError:
         print('Failed to retrieve item ID:', item_id)
         failed_requests.append(item_id)
         continue

   if len(failed_requests) > 0:
      print('The following item IDs could not be retrieved:', failed_requests)

   return items

items = get_item(item_ids)    

In [ ]:
# Check dataframe output

items_df = pd.DataFrame(items)

items_df.head()

In [ ]:
# Check for NaNs

items_df.isna().sum()

In [ ]:
# Check 404

items_df[items_df['name'] == 'UNKNOWN']

In [ ]:
# Count 404

items_df[items_df['name'] == 'UNKNOWN'].shape[0]

In [ ]:
# Save to file

items_df.to_csv('../data/items.csv', index = False)